In [ ]:
# !pip install torch torchvision matplotlib numpy pandas scikit-learn


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image

In [ ]:
selected_classes = [
    "apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio",
    "beignets", "bibimbap", "caesar_salad", "caprese_salad",
    "carrot_cake", "ceviche", "cheesecake", "chicken_curry",
    "chicken_quesadilla", "churros", "clam_chowder", "club_sandwich",
    "crab_cakes", "donuts", "eggs_benedict", "falafel"
]


base_path = "food-101/images"
filtered_paths = []
labels = []

for cls in selected_classes:
    class_path = os.path.join(base_path, cls)
    images = os.listdir(class_path)
    for img in images:
        filtered_paths.append(os.path.join(class_path, img))
        labels.append(cls)


data = pd.DataFrame({"image_path": filtered_paths, "label": labels})


train_df, test_df = train_test_split(data, test_size=0.2, stratify=data["label"], random_state=42)

In [ ]:

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


class FoodDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]["image_path"]
        label = self.dataframe.iloc[idx]["label"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, selected_classes.index(label)


train_dataset = FoodDataset(train_df, transform=train_transforms)
test_dataset = FoodDataset(test_df, transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of batches per epoch: {len(train_loader)}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())  
print(torch.cuda.get_device_name(0))  


class FoodClassifier(nn.Module):
    def __init__(self, num_classes):
        super(FoodClassifier, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

model = FoodClassifier(num_classes=len(selected_classes)).to(device)

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10
train_losses, test_accuracies = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    train_losses.append(running_loss / len(train_loader))
    
    
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    test_accuracy = correct / total
    test_accuracies.append(test_accuracy)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {test_accuracy:.4f}")


In [ ]:
plt.figure(figsize=(12, 5))


plt.subplot(1, 2, 1)
plt.plot(train_losses, label="Train Loss")
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()


plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label="Test Accuracy")
plt.title("Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.show()


In [ ]:
from google.colab import files
from PIL import Image


def estimate_portion_size(image_path):
    image = Image.open(image_path)
    width, height = image.size
    if width * height < 150000:
        return "Small"
    elif width * height < 300000:
        return "Medium"
    else:
        return "Large"


def predict(image_path, model, transform, classes):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)
    label = classes[predicted.item()]
    return label


uploaded = files.upload()


for file_name in uploaded.keys():
    image_path = file_name  


predicted_label = predict(image_path, model, test_transforms, selected_classes)
portion_size = estimate_portion_size(image_path)


print(f"Predicted: {predicted_label}, Portion Size: {portion_size}")